### "What just happened???"

Here we take an existing modflow model and setup a very complex parameterization system for arrays and boundary conditions.  All parameters are setup as multpliers: the original inputs from the modflow model are saved in separate files and during the forward run, they are multplied by the parameters to form new model inputs.  the forward run script ("forward_run.py") is also written.  And somewhat meaningful prior covariance matrix is constructed from geostatistical structures with out any additional arguements...oh yeah!

In [3]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu


In [4]:
nam_file = "freyberg.nam"
org_model_ws = "Freyberg_transient"
new_model_ws = "pest_setup"
# load the model, change dir and run once to get a hydmod output file and list file
m = flopy.modflow.Modflow.load(nam_file,model_ws=org_model_ws,check=False)
m.change_model_ws("temp",reset_external=True)
m.write_input()
try:
    m.run_model()
except():
    pass


changing model workspace...
   temp


Exception: The program mf2005.exe does not exist or is not executable.

You want some pilot points? We got that...how about one set of recharge multiplier pilot points applied to all stress periods? and sy in layer 1?

In [ ]:
pp_props = [["upw.sy",0], ["rch.rech",None]]

You want some constants (uniform value multipliers)?  We got that too....

In [ ]:
const_props = []
for iper in range(m.nper):
    const_props.append(["rch.rech",iper])
for k in range(m.nlay):
    const_props.append(["upw.hk",k])
    const_props.append(["upw.ss",k])


You want grid-scale parameter flexibility for hk in all layers? We got that too...and how about sy in layer 1 and vka in layer 2 while we are at it

In [ ]:
grid_props = [["upw.sy",0],["upw.vka",1]]
for k in range(m.nlay):
    grid_props.append(["upw.hk",k])
    

Some people like using zones...so we have those too

In [ ]:
zn_array = np.loadtxt(os.path.join("Freyberg_truth","hk.zones"))
plt.imshow(zn_array)

In [ ]:
zone_props = [["upw.ss",0]]
k_zone_dict = {k:zn_array for k in range(m.nlay)}

But wait, boundary conditions are uncertain too...Can we add some parameter to represent that uncertainty?  You know it!

In [ ]:
bc_props = [["drn.cond",None]] # drain conductance for all stress periods
for iper in range(m.nper):
    bc_props.append(["drn.elev",iper])
    bc_props.append(["wel.flux",iper])

In [ ]:
mfp_boss = pyemu.helpers.PstFromFlopyModel(nam_file,"temp",new_model_ws,
                                          pp_props=pp_props,bc_props=bc_props,
                                          zone_props=zone_props,grid_props=grid_props,
                                          const_props=const_props,k_zone_dict=k_zone_dict,
                                          remove_existing=True,pp_space=4)

In [ ]:
pst = mfp_boss.pst

In [ ]:
pst.npar,pst.nobs

In [ ]:
pst.template_files

In [ ]:
pst.instruction_files

In [ ]:
cov = pyemu.Cov.from_ascii(os.path.join(new_model_ws,mfp_boss.m.name+"_pest.pst.prior.cov"))

In [ ]:
cov = cov.x
cov[cov==0] = np.NaN
plt.imshow(cov)